### Funcion: get_features_num_regression

Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de regresión, es decir debe ser una variable numérica continua o discreta pero con alta cardinalidad, además de un argumento 'umbral_corr', de tipo float que debe estar entre 0 y 1 y una variable float "pvalue" cuyo valor debe ser por defecto "None".

La función debe devolver una lista con las columnas numéricas del dataframe cuya correlación con la columna designada por "target_col" sea superior en valor absoluto al valor dado por "umbral_corr". Además si la variable "pvalue" es distinta de None, sólo devolvera las columnas numéricas cuya correlación supere el valor indicado y además supere el test de hipótesis con significación mayor o igual a 1-pvalue.

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable numérica continua del dataframe.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway

def get_features_num_classification(df, target_col, pvalue=0.05):
    """
    Identifica columnas numéricas en un DataFrame que tienen un resultado significativo
    en la prueba ANOVA con respecto a una columna objetivo categórica.

    Argumentos:
    df (DataFrame): El DataFrame de entrada que contiene los datos.
    target_col (str): El nombre de la columna objetivo en el DataFrame. Esta debe ser 
                      una columna categórica con baja cardinalidad (10 o menos valores únicos).
    pvalue (float): El nivel de significancia para la prueba ANOVA. El valor por defecto es 0.05.

    Retorna:
    list: Una lista de nombres de columnas numéricas que tienen una relación significativa con 
          la columna objetivo según la prueba ANOVA.
          Retorna None si alguna de las comprobaciones de los argumentos de entrada falla, 
          e imprime un mensaje indicando la razón.
    """
    
    # Comprobación de que el DataFrame no está vacío
    if df.empty:
        print("El DataFrame está vacío.")
        return None
    
    # Comprobación de que target_col está en el DataFrame
    if target_col not in df.columns:
        print(f"La columna '{target_col}' no se encuentra en el DataFrame.")
        return None
    
    # Comprobación de que target_col es categórica con baja cardinalidad
    if not pd.api.types.is_categorical_dtype(df[target_col]) and not pd.api.types.is_object_dtype(df[target_col]):
        print(f"La columna '{target_col}' no es categórica.")
        return None
    
    if df[target_col].nunique() > 10:
        print(f"La columna '{target_col}' tiene demasiadas categorías (más de 10).")
        return None
    
    # Comprobación de que pvalue es un float y está en el rango correcto
    if not isinstance(pvalue, float) or not (0 < pvalue < 1):
        print("El valor de 'pvalue' debe ser un float entre 0 y 1.")
        return None
    
    # Filtrar las columnas numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Lista para almacenar las columnas que cumplen con el criterio
    significant_columns = []
    
    # Realizar el test ANOVA
    for col in numeric_cols:
        groups = [df[col][df[target_col] == category] for category in df[target_col].unique()]
        f_stat, p_val = f_oneway(*groups)
        if p_val >= (1 - pvalue):
            significant_columns.append(col)
    
    return significant_columns

In [ ]:
for col in columns:
            pd.to_numeric(df[col])
        if (float(pvalue) > 1) or (float(pvalue) < 0):
            return False

In [ ]:
#CHATGPT

import pandas as pd
import numpy as np
from scipy.stats import pearsonr

def get_features_num_regression(dataframe, target_col, umbral_corr, pvalue=None):
    """
    Obtiene columnas numéricas de un dataframe cuya correlación absoluta con una columna target
    es mayor que un umbral, opcionalmente evaluando significación estadística.

    Parámetros:
    - dataframe (pd.DataFrame): El dataframe de entrada.
    - target_col (str): Nombre de la columna numerica objetivo - continua o discreta - .
    - umbral_corr (float): Umbral para la correlación (debe estar entre 0 y 1).
    - pvalue (float, opcional): Valor p para el test de hipótesis. Si es None, no se considera.

    Retorna:
    - list: Lista con los nombres de las columnas que cumplen los criterios.
    - None: Si los parámetros de entrada son inválidos o no se encuentran columnas.
    """
    # Validaciones iniciales
    if not isinstance(dataframe, pd.DataFrame):
        print("Error: El argumento 'dataframe' debe ser un DataFrame de pandas.")
        return None
    if target_col not in dataframe.columns:
        print(f"Error: La columna '{target_col}' no existe en el dataframe.")
        return None
    if not np.issubdtype(dataframe[target_col].dtype, np.number):
        print(f"Error: La columna '{target_col}' debe ser numérica.")
        return None
    if not (0 <= umbral_corr <= 1):
        print("Error: 'umbral_corr' debe ser un valor entre 0 y 1.")
        return None
    if pvalue is not None and (not isinstance(pvalue, (float, int)) or not (0 < pvalue <= 1)):
        print("Error: 'pvalue' debe ser un valor float entre 0 y 1 o None.")
        return None

    # Filtrar columnas numéricas
    numeric_cols = dataframe.select_dtypes(include=[np.number]).columns.tolist()
    if target_col not in numeric_cols:
        print(f"Error: La columna '{target_col}' debe ser numérica.")
        return None

    numeric_cols.remove(target_col)  # Excluir la columna objetivo

    # Calcular correlaciones y filtrar
    selected_features = []
    for col in numeric_cols:
        correlation, p_val = pearsonr(dataframe[col].dropna(), dataframe[target_col].dropna())
        if abs(correlation) > umbral_corr:
            if pvalue is None or p_val < pvalue:
                selected_features.append(col)

    # Comprobar si se encontraron columnas
    if not selected_features:
        print("No se encontraron columnas que cumplan con los criterios.")
        return None

    return selected_features

# Ejemplo de uso
if __name__ == "__main__":
    # Crear un DataFrame de ejemplo
    data = {
        "feature1": [1, 2, 3, 4, 5],
        "feature2": [5, 4, 3, 2, 1],
        "feature3": [2, 2, 2, 2, 2],
        "target": [1.1, 2.1, 3.1, 4.1, 5.1],
    }
    df = pd.DataFrame(data)

    # Llamar a la función
    result = get_features_num_regression(df, target_col="target", umbral_corr=0.9, pvalue=0.05)
    print("Columnas seleccionadas:", result)


#ChatGPT
Explicación:
1. Validaciones de entrada:
. Verifica que dataframe sea un pd.DataFrame.
- Valida que target_col exista en el dataframe y sea una columna numérica.
- Asegura que umbral_corr esté entre 0 y 1.
- Si pvalue no es None, valida que esté entre 0 y 1.
2. Filtrar columnas numéricas:
- Selecciona columnas numéricas usando select_dtypes.
3. Calcular correlación y p-value:
- Utiliza pearsonr para calcular la correlación y el p-valor entre cada columna numérica y la columna objetivo.
- Aplica los filtros según el umbral de correlación y el p-valor (si se especifica).
4. Resultados:
- Si no se encuentran columnas que cumplan los criterios, devuelve None e imprime un mensaje.

In [ ]:
#CLAUDE
# Ejemplo de uso
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
from scipy import stats

def get_features_num_regression(df, target_col, umbral_corr, pvalue=None):
    """
    Selecciona features numéricas basadas en su correlación con la variable target.
    
    Args:
        df (pandas.DataFrame): DataFrame de entrada
        target_col (str): Nombre de la columna target
        umbral_corr (float): Umbral de correlación (valor absoluto) entre 0 y 1
        pvalue (float, optional): Nivel de significación para el test de hipótesis
        
    Returns:
        list: Lista de columnas que cumplen los criterios o None si hay error
    """
    # Validación de entrada del DataFrame
    if not isinstance(df, pd.DataFrame):
        print("Error: El primer argumento debe ser un DataFrame")
        return None
    
    # Validación de target_col
    if not isinstance(target_col, str):
        print("Error: target_col debe ser una cadena de texto")
        return None
        
    if target_col not in df.columns:
        print(f"Error: La columna {target_col} no existe en el DataFrame")
        return None
    
    # Verificar si target_col es numérica
    if not np.issubdtype(df[target_col].dtype, np.number):
        print(f"Error: La columna {target_col} debe ser numérica")
        return None
        
    # Verificar si target_col tiene suficientes valores únicos (alta cardinalidad)
    n_unique = df[target_col].nunique()
    if n_unique < 10:  # umbral arbitrario para considerar alta cardinalidad
        print(f"Error: La columna {target_col} debe tener alta cardinalidad (tiene {n_unique} valores únicos)")
        return None
    
    # Validación de umbral_corr
    if not isinstance(umbral_corr, (int, float)):
        print("Error: umbral_corr debe ser un número")
        return None
        
    if umbral_corr < 0 or umbral_corr > 1:
        print("Error: umbral_corr debe estar entre 0 y 1")
        return None
    
    # Validación de pvalue si está presente
    if pvalue is not None:
        if not isinstance(pvalue, (int, float)):
            print("Error: pvalue debe ser un número")
            return None
            
        if pvalue <= 0 or pvalue > 1:
            print("Error: pvalue debe estar entre 0 y 1")
            return None
    
    # Obtener columnas numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols.remove(target_col)  # Excluir la columna target
    
    selected_features = []
    
    for col in numeric_cols:
        # Calcular correlación y p-valor
        correlation, p_val = stats.pearsonr(df[target_col].dropna(), df[col].dropna())
        
        # Verificar si la correlación supera el umbral (en valor absoluto)
        if abs(correlation) > umbral_corr:
            # Si pvalue es None, añadir la columna
            if pvalue is None:
                selected_features.append(col)
            # Si pvalue no es None, verificar también la significación estadística
            elif p_val <= pvalue:
                selected_features.append(col)
    
    return selected_features



# Crear datos de ejemplo
df = pd.DataFrame({
    'target': np.random.normal(0, 1, 1000),
    'feature1': np.random.normal(0, 1, 1000),
    'feature2': np.random.normal(0, 1, 1000)
})

# Llamar a la función
selected_features = get_features_num_regression(
    df=df,
    target_col='target',
    umbral_corr=0.1,
    pvalue=0.05
)

#CLAUDE
Validaciones exhaustivas de los argumentos de entrada:

Verifica que df sea un DataFrame
Comprueba que target_col sea una cadena y exista en el DataFrame
Valida que target_col sea numérica y tenga alta cardinalidad
Verifica que umbral_corr sea un número entre 0 y 1
Si pvalue está presente, verifica que sea un número entre 0 y 1


Funcionalidad principal:

Identifica todas las columnas numéricas del DataFrame
Calcula la correlación de Pearson y p-valor para cada columna
Selecciona features basándose en el umbral de correlación
Si pvalue está especificado, añade el filtro de significación estadística


Manejo de errores:

Retorna None y muestra mensajes explicativos cuando hay problemas
Maneja apropiadamente los valores NA en los cálculos de correlación

In [ ]:
#GEMINI

import pandas as pd
import numpy as np

def get_features_num_regression(df, target_col, umbral_corr, pvalue=None):
    """
    Identifica las columnas numéricas de un DataFrame con una correlación
    suficientemente alta con una columna objetivo y, opcionalmente, un p-valor.

    Args:
        df: DataFrame de entrada.
        target_col: Nombre de la columna objetivo (numérica).
        umbral_corr: Umbral mínimo de correlación en valor absoluto.
        pvalue: Valor p máximo para considerar una correlación significativa.

    Returns:
        Lista de nombres de columnas que cumplen los criterios, o None si hay errores.
    """

    # Comprobaciones de entrada
    if not isinstance(df, pd.DataFrame):
        print("El primer argumento debe ser un DataFrame de pandas.")
        return None
    if target_col not in df.columns:
        print(f"La columna '{target_col}' no existe en el DataFrame.")
        return None
    if not pd.api.types.is_numeric_dtype(df[target_col]):
        print(f"La columna '{target_col}' debe ser numérica.")
        return None
    if not 0 <= umbral_corr <= 1:
        print("El umbral de correlación debe estar entre 0 y 1.")
        return None
    if pvalue is not None and not 0 <= pvalue <= 1:
        print("El valor p debe estar entre 0 y 1.")
        return None

    # Seleccionar columnas numéricas y calcular correlaciones
    num_cols = df.select_dtypes(include=['number']).columns
    corr_matrix = df[num_cols].corrwith(df[target_col])

    # Filtrar por umbral de correlación
    selected_cols = corr_matrix[(corr_matrix.abs() > umbral_corr)].index

    # Filtrar por p-valor si se especifica
    if pvalue is not None:
        # Aquí necesitarías realizar un test de hipótesis para obtener los p-valores.
        # Puedes usar scipy.stats.pearsonr o otras funciones similares.
        # Por simplicidad, asumimos que tienes una función 'get_pvalues' que calcula los p-valores:
        pvalues = get_pvalues(df, target_col, num_cols)
        selected_cols = selected_cols[pvalues[selected_cols] <= pvalue]

    return list(selected_cols)

#GEMINI

Explicación:

Comprobaciones: Se verifican los tipos de datos, la existencia de la columna objetivo y los rangos de los valores de entrada.
Selección de columnas numéricas: Se seleccionan todas las columnas numéricas del DataFrame.
Cálculo de correlaciones: Se calcula la matriz de correlación entre las columnas numéricas y la columna objetivo.
Filtrado por umbral de correlación: Se seleccionan las columnas cuya correlación en valor absoluto supera el umbral especificado.
Filtrado por p-valor (opcional): Si se proporciona un valor de p, se filtran las columnas según los resultados del test de hipótesis. Nota: Aquí se asume que tienes una función get_pvalues que calcula los p-valores. Necesitarás implementarla utilizando una biblioteca como scipy.stats.
Importante:

Test de hipótesis: La parte del código que filtra por p-valor es un ejemplo. Necesitarás implementar la función get_pvalues para realizar el test de hipótesis adecuado (por ejemplo, usando scipy.stats.pearsonr).
Otras consideraciones:
Multicolinealidad: Si hay alta multicolinealidad entre las variables, podrías considerar técnicas de selección de características como el análisis de componentes principales (PCA) o el LASSO.
Normalidad: Para el test de hipótesis, es recomendable verificar si los datos se distribuyen aproximadamente de forma normal.
Heterocedasticidad: Si hay heterocedasticidad, los resultados del test de hipótesis podrían no ser válidos.
Ejemplo de uso:

Python

# Suponiendo que tienes un DataFrame df y una función get_pvalues implementada:
selected_features = get_features_num_regression(df, 'precio', 0.5, 0.05)
print(selected_features)
Esta función te proporcionará una base sólida para seleccionar las características más relevantes para un modelo de regresión, teniendo en cuenta tanto la correlación como la significancia estadística.

In [ ]:
#COPILOT
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

def get_features_num_regression(df: pd.DataFrame, target_col: str, umbral_corr: float, pvalue: float = None) -> list:
    # Verificar que target_col existe en el dataframe
    if target_col not in df.columns:
        print("Error: La columna objetivo no existe en el dataframe.")
        return None
    
    # Verificar que target_col es una columna numérica continua
    if not np.issubdtype(df[target_col].dtype, np.number):
        print("Error: La columna objetivo no es una variable numérica continua.")
        return None
    
    # Verificar que umbral_corr está entre 0 y 1
    if not (0 <= umbral_corr <= 1):
        print("Error: El umbral de correlación debe estar entre 0 y 1.")
        return None
    
    # Lista para almacenar las columnas que cumplen con los criterios
    selected_columns = []

    # Iterar sobre todas las columnas numéricas del dataframe
    for col in df.select_dtypes(include=np.number).columns:
        if col != target_col:
            corr = df[target_col].corr(df[col])
            if abs(corr) > umbral_corr:
                if pvalue is not None:
                    _, pval = pearsonr(df[target_col], df[col])
                    if pval >= (1 - pvalue):
                        selected_columns.append(col)
                else:
                    selected_columns.append(col)
    
    return selected_columns


#COPILOT 
Detalles:
Verificaciones iniciales:

Comprueba que target_col exista en el dataframe.

Verifica que target_col sea una columna numérica continua.

Asegura que umbral_corr esté entre 0 y 1.

Cálculo de correlaciones:

Itera sobre todas las columnas numéricas y calcula la correlación con target_col.

Añade las columnas a la lista si cumplen con los criterios de correlación y significancia estadística (si pvalue no es None).

In [4]:
def get_features_num_regression(df, target_col, umbral_corr, pvalue=None):
    '''
    Selecciona features numéricas basadas en su correlación con la variable target.
    
    Args:
        df (pandas.DataFrame): DataFrame de entrada
        target_col (str): Nombre de la columna target
        umbral_corr (float): Umbral de correlación (valor absoluto) entre 0 y 1
        pvalue (float, optional): Nivel de significación para el test de hipótesis
        
    Returns:
        list: Lista de columnas que cumplen los criterios o None si hay error
    '''
    ## Validaciones de entrada
    # Verifica que df sea un DataFrame
    if not isinstance(df, pd.DataFrame):
        print("Error: El primer argumento debe ser un DataFrame")
        return None
    # Verifica que target_col sea una cadena y exista en el DataFrame
    if target_col not in df.columns:
        print("Error: La columna objetivo no existe en el dataframe.")
        return None
    if not isinstance(target_col, str):
        print("Error: target_col debe ser una cadena de texto")
        return None
    # Verifica que target_col sea numérica
    if not np.issubdtype(df[target_col].dtype, np.number):
        print(f"Error: La columna {target_col} debe ser numérica")
        return None
    # Verificar si target_col tiene suficientes valores únicos (alta cardinalidad)
    n_unique = df[target_col].nunique()
    if n_unique < 30:  # umbral arbitrario para considerar alta cardinalidad
        print(f"Error: La columna {target_col} debe tener alta cardinalidad (tiene {n_unique} valores únicos)")
        return None
    
    # Verifica que umbral_corr está entre 0 y 1
    if not (0 <= umbral_corr <= 1):
        print("Error: El umbral de correlación debe estar entre 0 y 1.")
        return None
    
    # Validación de pvalue si está presente
    if pvalue is not None and not 0 <= pvalue <= 1:
        print("El valor p debe estar entre 0 y 1.")
        return None
    

    # Lista para almacenar las columnas que cumplen con los criterios
    selected_columns = []

    # Iterar sobre todas las columnas numéricas del dataframe 
    # excluiendo la target y las numericas con cardinalidad baja que pueden ser consideradas categoricas 
    for col in df.select_dtypes(include=np.number).columns:
        if col != target_col and df[col].nunique() >= 15:
            corr = df[target_col].corr(df[col])
            if abs(corr) > umbral_corr:
                if pvalue is not None:
                    _, pval = pearsonr(df[target_col], df[col])
                    if pval >= (1 - pvalue):
                        selected_columns.append(col)
                else:
                    selected_columns.append(col)

    return selected_columns


1. agregar un análisis de multicolinealidad a la función get_features_num_regression. La multicolinealidad puede afectar significativamente los resultados de un modelo de regresión, por lo que es importante identificarla y abordarla.
2. añadir analisis de normalidad??